# Adaptively Sampled MPC

This is a simple demonstration of an adaptively sampled region quadtree used to approximate the solutions to a model-predictive control problem. 

In [1]:
using RegionTrees
import StaticArrays: SVector
using Plots
Plots.gr()

Plots.GRBackend()

In [23]:
# This module implements our MPC code, which consists of a 1-dimensional
# double-integrator system, forward-euler time-stepping dynamics, and a 
# 10 step prediction window. 

module mpc

using JuMP
using Gurobi
using StaticArrays
using Interpolations
import RegionTrees
import RegionTrees: AbstractRefinery, needs_refinement, refine_data, center, vertices

# Solve the MPC problem from a given initial position and velocity
function run_mpc(q0, v0)
    model = Model(solver=GurobiSolver(OutputFlag=0))
    num_time_steps = 10
    dt = 0.1
    u_limit = 3
    C_q = 100
    c_vfinal = 100
    C_u = 1

    @variable model q[1:num_time_steps]
    @variable model v[1:num_time_steps]
    @variable model u[1:num_time_steps]

    @constraint model [i=2:num_time_steps] q[i] == q[i-1] + v[i-1] * dt
    @constraint model [i=2:num_time_steps] v[i] == v[i-1] + u[i-1] * dt
    @constraint model u .<= u_limit
    @constraint model u .>= -u_limit
    @constraint model q[1] == q0
    @constraint model v[1] == v0

    @objective model Min C_q * sum{q[i]^2, i=1:num_time_steps} + c_vfinal * v[end]^2 + C_u * sum{u[i]^2, i=1:num_time_steps}
    solve(model)

    getvalue(q), getvalue(v), getvalue(u)
end

# The MPCRefinery provides enough behavior to implement the 
# RegionTrees AdaptiveSampling interface, which lets us generate
# a quadtree of initial states and their corresponding MPC solutions.
immutable MPCRefinery <: AbstractRefinery
end

function evaluate(cell, point)
    p = (point - cell.boundary.origin) ./ cell.boundary.widths
    cell.data[p[1] + 1, p[2] + 1]
end

# A cell in the quadtree needs refinement if its interpolated solution
# derived from its vertices is not a good fit for the true MPC solution
# at its center. 
function needs_refinement(::MPCRefinery, cell)
    x = center(cell)
    value_interp = evaluate(cell, x)
    value_true = run_mpc(x[1], x[2])[3]
    !isapprox(value_interp[1], value_true[1], rtol=1e-1, atol=1e-1)
end

# The data element associated with a cell is a bilinear interpolation
# of the MPC function evaluated at the vertices of the cell.
function refine_data(::MPCRefinery, boundary)
    f = v -> run_mpc(v[1], v[2])[3]
    interpolate(f.(vertices(boundary)),
                BSpline(Linear()), OnGrid())
end

# Needed to let us interpolate directly on arrays of control inputs
Base.one{T, N}(::Type{Array{T, N}}) = one(T)

end

mpc

In [24]:
# Simulate a given control function for the double integerator
# model.
function simulate(controller, q0, v0, dt, timespan)
    num_time_steps = timespan / dt
    qs = [q0]
    vs = [v0]
    ts = [0.0]
    q = q0
    v = v0
    for t in 0:dt:(timespan)
        u = controller(t, q, v)
        q += v * dt
        v += u * dt
        push!(qs, q)
        push!(vs, v)
        push!(ts, t)
    end
    ts, qs, vs
end
        

simulate (generic function with 1 method)

In [25]:
controller = (t, q, v) -> begin
    q, v, u = mpc.run_mpc(q, v)
    u[1]
end

(::#7) (generic function with 1 method)

In [26]:
t, q, v = simulate(controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  8.59724e-5,5.36258e-5,2.83979e-5,9.84092e-6,-2.88847e-6,-1.08171e-5,-1.50039e-5,-1.64406e-5,-1.59925e-5,-1.43722e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.000323466,-0.000252279,-0.00018557,-0.000127294,-7.9286e-5,-4.18688e-5,-1.43665e-5,4.48084e-6,1.6203e-5,2.23757e-5])

In [27]:
# State space portrait of the solution, starting from q=10, v=0
plot(q, v, xlim=(-10, 10), ylim=(-10, 10), xlabel="q", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 q 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,186.806 596.063,192.292 595.238,197.778 593.587,203.264 591.111,208.75 587.81,214.236 583.684,219.722 578.732,225.208 572.955,230.694 566.353,236.18 
 558.926,241.666 550.673,247.152 541.595,252.638 531.692,258.124 520.963,263.61 509.409,269.096 497.03,274.582 483.826,280.069 469.796,285.555 454.941,291.041 
 439.261,296.527 422.756,292.534 406.851,287.048 391.771,281.562 377.517,276.076 364.088,270.59 351.484,265.104 339.706,259.618 328.752,254.132 318.624,248.646 
 309.322,243.16 300.844,237.674 293.192,232.188 286.365,226.702 280.364,221.216 275.187,215.73 270.836,210.244 267.31,204.757 264.61,199.271 262.735,193.785 
 261.685,188.299 261.46,182.813 262.061,177.327 263.486,171.841 265.738,166.355 268.814,160.869 272.716,155.383 277.443,149.897 282.995,144.411 289.372,142.223 
 296.079,144.539 302.437,149.805 308.003,155.291 312.744,160.777 316.659,166.263 319.749,171.749 322.014,177.235 323.454,182.038 324.171,185.518 324.365,187.859 
 324.206,189.273 323.835,189.975 323.358,190.162 322.853,190.002 322.373,189.632 321.947,189.16 321.593,188.661 321.314,188.186 321.107,187.766 320.962,187.416 
 320.87,187.141 320.82,186.936 320.8,186.794 320.802,186.704 320.817,186.654 320.84,186.635 320.866,186.637 320.891,186.652 320.914,186.675 320.934,186.7 
 320.949,186.725 320.961,186.748 320.97,186.767 320.976,186.783 320.979,186.795 320.981,186.804 320.981,186.809 320.981,186.812 320.979,186.814 320.978,186.814 
 320.977,186.814 320.976,186.813 320.975,186.811 320.974,186.81 320.973,186.809 320.972,186.808 320.972,186.807 320.972,186.806 320.972,186.806 320.972,186.805 
 320.972,186.805 320.972,186.805 
 "/>

In [28]:
plot(t, q, xlabel="t", ylabel="q", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 t 
 
 
 q 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,3.93701 54.7703,3.93701 60.1833,4.83907 65.5962,6.64319 71.0091,9.34938 76.422,12.9576 81.835,17.4679 87.2479,22.8803 92.6608,29.1947 98.0738,36.4112 
 103.487,44.5298 108.9,53.5504 114.313,63.4731 119.725,74.2979 125.138,86.0247 130.551,98.6535 135.964,112.184 141.377,126.617 146.79,141.953 152.203,158.19 
 157.616,175.329 163.029,193.37 168.442,210.755 173.855,227.238 179.268,242.818 184.681,257.497 190.094,271.273 195.506,284.148 200.919,296.12 206.332,307.191 
 211.745,317.359 217.158,326.625 222.571,334.989 227.984,342.451 233.397,349.012 238.81,354.67 244.223,359.425 249.636,363.279 255.049,366.231 260.462,368.281 
 265.874,369.428 271.287,369.674 276.7,369.018 282.113,367.459 287.526,364.999 292.939,361.636 298.352,357.371 303.765,352.204 309.178,346.136 314.591,339.165 
 320.004,331.834 325.417,324.884 330.83,318.8 336.243,313.619 341.655,309.339 347.068,305.961 352.481,303.485 357.894,301.912 363.307,301.128 368.72,300.916 
 374.133,301.089 379.546,301.495 384.959,302.016 390.372,302.568 395.785,303.094 401.198,303.558 406.611,303.946 412.023,304.251 417.436,304.477 422.849,304.635 
 428.262,304.736 433.675,304.791 439.088,304.812 444.501,304.811 449.914,304.794 455.327,304.769 460.74,304.741 466.153,304.713 471.566,304.688 476.979,304.667 
 482.392,304.649 487.804,304.636 493.217,304.627 498.63,304.62 504.043,304.617 509.456,304.615 514.869,304.615 520.282,304.615 525.695,304.616 531.108,304.618 
 536.521,304.619 541.934,304.621 547.347,304.622 552.76,304.623 558.173,304.624 563.585,304.624 568.998,304.624 574.411,304.625 579.824,304.625 585.237,304.625 
 590.65,304.625 596.063,304.625 
 "/>

In [29]:
plot(t, v, xlabel="t", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 t 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,109.608 54.7703,122.612 60.1833,135.615 65.5962,148.618 71.0091,161.622 76.422,174.625 81.835,187.628 87.2479,200.631 92.6608,213.635 98.0738,226.638 
 103.487,239.641 108.9,252.645 114.313,265.648 119.725,278.651 125.138,291.654 130.551,304.658 135.964,317.661 141.377,330.664 146.79,343.668 152.203,356.671 
 157.616,369.674 163.029,360.211 168.442,347.208 173.855,334.205 179.268,321.201 184.681,308.198 190.094,295.195 195.506,282.192 200.919,269.188 206.332,256.185 
 211.745,243.182 217.158,230.178 222.571,217.175 227.984,204.172 233.397,191.169 238.81,178.165 244.223,165.162 249.636,152.159 255.049,139.155 260.462,126.152 
 265.874,113.149 271.287,100.146 276.7,87.1423 282.113,74.139 287.526,61.1357 292.939,48.1324 298.352,35.1291 303.765,22.1258 309.178,9.12256 314.591,3.93701 
 320.004,9.42701 325.417,21.9078 330.83,34.9111 336.243,47.9144 341.655,60.9177 347.068,73.921 352.481,86.9243 357.894,98.3093 363.307,106.557 368.72,112.105 
 374.133,115.457 379.546,117.121 384.959,117.563 390.372,117.184 395.785,116.309 401.198,115.189 406.611,114.005 412.023,112.879 417.436,111.884 422.849,111.056 
 428.262,110.403 433.675,109.919 439.088,109.582 444.501,109.368 449.914,109.25 455.327,109.205 460.74,109.209 466.153,109.245 471.566,109.299 476.979,109.359 
 482.392,109.418 487.804,109.472 493.217,109.518 498.63,109.555 504.043,109.583 509.456,109.604 514.869,109.617 520.282,109.625 525.695,109.628 531.108,109.629 
 536.521,109.628 541.934,109.625 547.347,109.622 552.76,109.619 558.173,109.616 563.585,109.614 568.998,109.612 574.411,109.61 579.824,109.609 585.237,109.608 
 590.65,109.608 596.063,109.607 
 "/>

# Approximating the solutions space
Now that we've written down and tested the true MPC solution, we can try to approximate it. The AdaptiveSampling() function will generate a quadtree by iteratively refining each cell in the space until the solutions to the MPC problem within that cell are well-approximated by the interpolation. 

In [30]:
root = RegionTrees.AdaptiveSampling(mpc.MPCRefinery(), SVector(-9, -9), SVector(19, 19))

Cell: RegionTrees.HyperRectangle{2,Float64}([-9.0,-9.0],[19.0,19.0])

In [31]:
# Now we can plot each region in the quadtree. Note that we end up with
# a lot of detail along the switching surface from maximum acceleration
# to maximum deceleration
plt = plot(xlim=(-10, 10), ylim=(-10, 10), legend=nothing, grid=false)
queue = [root]
i = 0
while !isempty(queue) 
    i += 1
    cell = pop!(queue)
    v = hcat(collect(RegionTrees.vertices(cell.boundary))...)
    plot!(v[1,[1,2,4,3,1]], v[2,[1,2,4,3,1]])
    if !RegionTrees.isleaf(cell)
        append!(queue, RegionTrees.children(cell))
    end
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10

# Using the Approximate Solution
We can use the quadtree to produce an approximate controller for our system. Given the current state, we can look up the cell in the quadtree which encompasses that state, then use that cell's interpolation to find our control tape. 

In [32]:
approx_controller = (t, q, v) -> begin
    x = [q, v]
    leaf = RegionTrees.findleaf(root, x)
    u = mpc.evaluate(leaf, x)
    u[1]
end

(::#9) (generic function with 1 method)

In [33]:
t, q, v = simulate(approx_controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  0.000312392,0.000214144,0.000133246,7.02246e-5,2.3927e-5,-7.7792e-6,-2.7479e-5,-3.78317e-5,-4.1323e-5,-4.01199e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.000982478,-0.00080898,-0.000630219,-0.000462976,-0.000317062,-0.000196998,-0.000103527,-3.49131e-5,1.20305e-5,4.11549e-5])

In [34]:
# The results are similar to, but not quite as good as, the 
# values obtained from the exact MPC solution. 
plot(q, v, xlim=(-10, 10), ylim=(-10, 10), xlabel="q", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 q 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,186.806 596.063,192.292 595.238,197.778 593.587,203.264 591.111,208.75 587.81,214.236 583.684,219.722 578.732,225.208 572.955,230.694 566.353,236.18 
 558.926,241.666 550.673,247.152 541.595,252.638 531.692,258.124 520.963,263.61 509.409,269.096 497.03,274.582 483.826,280.069 469.796,285.555 454.941,291.041 
 439.261,296.527 422.756,292.673 406.83,287.187 391.73,281.701 377.455,276.215 364.005,270.729 351.38,265.243 339.581,259.756 328.607,254.704 318.393,249.218 
 309.004,243.732 300.44,238.246 292.702,232.76 285.789,227.274 279.701,221.788 274.439,216.302 270.002,210.816 266.39,205.33 263.603,199.844 261.642,194.358 
 260.506,188.872 260.195,183.385 260.71,177.899 262.049,172.413 264.214,167.147 267.172,162.119 270.885,157.342 275.317,151.856 280.575,146.37 286.658,142.169 
 293.372,143.029 299.958,146.977 305.949,151.72 311.227,155.549 315.929,159.384 320.054,164.83 323.36,170.316 325.84,175.155 327.593,179.437 328.701,184.709 
 329.017,189.498 328.612,191.998 327.831,193.531 326.819,194.224 325.703,194.23 324.586,193.714 323.547,192.837 322.64,191.745 321.896,190.564 321.331,189.398 
 320.941,188.324 320.713,187.636 320.588,187.126 320.54,186.771 320.545,186.547 320.584,186.424 320.641,186.377 320.706,186.383 320.769,186.422 320.827,186.479 
 320.876,186.542 320.916,186.605 320.946,186.662 320.967,186.711 320.981,186.75 320.99,186.78 320.994,186.801 320.994,186.815 320.993,186.823 320.99,186.827 
 320.987,186.828 320.984,186.826 320.981,186.824 320.978,186.82 320.976,186.817 320.974,186.814 320.973,186.811 320.972,186.809 320.971,186.807 320.971,186.806 
 320.971,186.805 320.971,186.805 
 "/>